In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from model_points import theta_polynomial, get_MPs, compute_stat_MP, compute_stat_Sn, quantile_Sn, pmf_Sn, CF_approximation

# Interaction with the smart contract

This notebook is a python check of the scenario of transactions submitted to the smart contract. 

We first define a couple of parameters


In [36]:
contract_owner_eth_address, investor_eth_address, policyholder_eth_address = "0xE8e79B8B8c0481fa33a8E0fcA902ad5754BfE1C3", "0x2CF8ed1664616483c12Ef3113f6F62E68f1a810A", "0xd34a37613A382bA503f1599F514C9788dF3659C4"
contract_owner_balance, investor_balance, policyholder_balance = [0.482], [0.356], [0.364]
# risk level parameter
alpha_MCR, alpha_SCR = 0.85, 0.995
alphas = [alpha_MCR, alpha_SCR]
# safety margin parameter
eta = 0.1
n_MP = 5
# surplus and balance processeses
Xt, Bt = [0], [0]
# Number of contracts, number of investors, investors' token holding, number of model points
Nt, At, Yt, Mt = [0], [0], [0], [0]
TotalSupply, rt = [0], [1]
# mean, variance, skewness and kurtosis processes
Mut, Vart, gammat_1, gammat_2 = [0], [0], [0], [0]
# MCR and SCR processes
MCRt, SCRt = [0], [0]
MCRt_CF2, SCRt_CF2 = [0], [0]
MCRt_CF3, SCRt_CF3 = [0], [0]
# Initialize the portfolio
portfolio = pd.DataFrame(columns=['contract_ID','S', 'T', 'station', 'Q_bar', 'l', 'theta'])


In [37]:
def add_contract(S, T, station, Q_bar, l):
    theta = theta_polynomial(T, station)
    portfolio.loc[len(portfolio)] = [len(portfolio)+1, S, T, station, Q_bar, l, theta]
    MPs = get_MPs(portfolio)

    # surplus and balance processeses
    cp = theta * (1 + eta) * l
    Xt.append(Xt[-1]), Bt.append(Bt[-1] + cp), At.append(At[-1]), TotalSupply.append(TotalSupply[-1])
    Nt.append(Nt[-1]+1), Mt.append(len(MPs))
    rt.append(rt[-1])

    if(Mt[-1] < n_MP):
        S_Zs, p_Zs = [d['S_Z'] for d in MPs], [d['p_Z'] for d in MPs]
        stats_mat = np.array([compute_stat_MP(S_Z, p_Z) for S_Z, p_Z in zip(S_Zs, p_Zs)])
        stats_Sn = compute_stat_Sn(stats_mat)
        pmf_Sn_fft = pmf_Sn(S_Zs, p_Zs)

        Q_mat = np.array([CF_approximation(stats_Sn, alpha) for alpha in alphas])
        Q_Sn = np.array([quantile_Sn(pmf_Sn_fft, alpha) for alpha in alphas])
        Q_mat[0,0] - stats_Sn[0] * (1 + eta), Q_mat[1,0] - stats_Sn[0] * (1 + eta)

        Mut.append(stats_Sn[0]), Vart.append(stats_Sn[1]), gammat_1.append(stats_Sn[2]), gammat_2.append(stats_Sn[3])
        
        MCRt.append(portfolio.l.sum()), SCRt.append(portfolio.l.sum())
        MCRt_CF2.append(portfolio.l.sum()), SCRt_CF2.append(portfolio.l.sum())
        MCRt_CF3.append(portfolio.l.sum()), SCRt_CF3.append(portfolio.l.sum())
    else:
        S_Zs, p_Zs = [d['S_Z'] for d in MPs], [d['p_Z'] for d in MPs]
        stats_mat = np.array([compute_stat_MP(S_Z, p_Z) for S_Z, p_Z in zip(S_Zs, p_Zs)])
        stats_Sn = compute_stat_Sn(stats_mat)
        pmf_Sn_fft = pmf_Sn(S_Zs, p_Zs)

        Q_mat = np.array([CF_approximation(stats_Sn, alpha) for alpha in alphas])
        Q_Sn = np.array([quantile_Sn(pmf_Sn_fft, alpha) for alpha in alphas])
        Q_mat[0,0] - stats_Sn[0] * (1 + eta), Q_mat[1,0] - stats_Sn[0] * (1 + eta)

        Mut.append(stats_Sn[0]), Vart.append(stats_Sn[1]), gammat_1.append(stats_Sn[2]), gammat_2.append(stats_Sn[3])

        MCRt.append(Q_Sn[0]- stats_Sn[0] * (1 + eta)), SCRt.append(Q_Sn[1]- stats_Sn[0] * (1 + eta))
        MCRt_CF2.append(Q_mat[0,0] - stats_Sn[0] * (1 + eta)), SCRt_CF2.append(Q_mat[1,0] - stats_Sn[0] * (1 + eta))
        MCRt_CF3.append(Q_mat[0,1] - stats_Sn[0] * (1 + eta)), SCRt_CF3.append(Q_mat[1,1] - stats_Sn[0] * (1 + eta))
    
def remove_contract(contract_ID, Q_obs):
    global portfolio
    contract_details = portfolio.loc[portfolio['contract_ID'] == contract_ID, ['S', 'T', 'station', 'Q_bar', 'l', 'theta']].iloc[0]
    S, T, station, Q_bar, l, theta = contract_details['S'], contract_details['T'], contract_details['station'], contract_details['Q_bar'], contract_details['l'], contract_details['theta']
    cp = theta * (1 + eta) * l
    if Q_obs < Q_bar:
        Xt.append(Xt[-1] + cp), Bt.append(Bt[-1]), At.append(At[-1]), TotalSupply.append(TotalSupply[-1])
    else:
        Xt.append(Xt[-1] + cp - l), Bt.append(Bt[-1] - l), At.append(At[-1]), TotalSupply.append(TotalSupply[-1])
    portfolio = portfolio[portfolio['contract_ID'] != contract_ID].reset_index(drop=True)
    MPs = get_MPs(portfolio)
    Nt.append(Nt[-1]-1), Mt.append(len(MPs))
    rt.append(Xt[-1] / TotalSupply[-1])
    if(Mt[-1] < n_MP):
        S_Zs, p_Zs = [d['S_Z'] for d in MPs], [d['p_Z'] for d in MPs]
        stats_mat = np.array([compute_stat_MP(S_Z, p_Z) for S_Z, p_Z in zip(S_Zs, p_Zs)])
        stats_Sn = compute_stat_Sn(stats_mat)
        pmf_Sn_fft = pmf_Sn(S_Zs, p_Zs)

        Q_mat = np.array([CF_approximation(stats_Sn, alpha) for alpha in alphas])
        Q_Sn = np.array([quantile_Sn(pmf_Sn_fft, alpha) for alpha in alphas])
        Q_mat[0,0] - stats_Sn[0] * (1 + eta), Q_mat[1,0] - stats_Sn[0] * (1 + eta)

        Mut.append(stats_Sn[0]), Vart.append(stats_Sn[1]), gammat_1.append(stats_Sn[2]), gammat_2.append(stats_Sn[3])
        
        MCRt.append(portfolio.l.sum()), SCRt.append(portfolio.l.sum())
        MCRt_CF2.append(portfolio.l.sum()), SCRt_CF2.append(portfolio.l.sum())
        MCRt_CF3.append(portfolio.l.sum()), SCRt_CF3.append(portfolio.l.sum())
    else:
        S_Zs, p_Zs = [d['S_Z'] for d in MPs], [d['p_Z'] for d in MPs]
        stats_mat = np.array([compute_stat_MP(S_Z, p_Z) for S_Z, p_Z in zip(S_Zs, p_Zs)])
        stats_Sn = compute_stat_Sn(stats_mat)
        pmf_Sn_fft = pmf_Sn(S_Zs, p_Zs)

        Q_mat = np.array([CF_approximation(stats_Sn, alpha) for alpha in alphas])
        Q_Sn = np.array([quantile_Sn(pmf_Sn_fft, alpha) for alpha in alphas])
        Q_mat[0,0] - stats_Sn[0] * (1 + eta), Q_mat[1,0] - stats_Sn[0] * (1 + eta)

        Mut.append(stats_Sn[0]), Vart.append(stats_Sn[1]), gammat_1.append(stats_Sn[2]), gammat_2.append(stats_Sn[3])

        MCRt.append(Q_Sn[0]- stats_Sn[0] * (1 + eta)), SCRt.append(Q_Sn[1]- stats_Sn[0] * (1 + eta))
        MCRt_CF2.append(Q_mat[0,0] - stats_Sn[0] * (1 + eta)), SCRt_CF2.append(Q_mat[1,0] - stats_Sn[0] * (1 + eta))
        MCRt_CF3.append(Q_mat[0,1] - stats_Sn[0] * (1 + eta)), SCRt_CF3.append(Q_mat[1,1] - stats_Sn[0] * (1 + eta))




Nous définissons les évènements suivant:

1. Investor contributes $x$ and receives $y = x \cdot r_t$ protocol tokens

In [38]:
x = 100
y = rt[-1] * x
Yt.append(Yt[-1] + y), Xt.append(Xt[-1] + x), Bt.append(Bt[-1] + x), At.append(At[-1] + 1), TotalSupply.append(TotalSupply[-1] + y)
Nt.append(Nt[-1]), Mt.append(Mt[-1])
Mut.append(Mut[-1]), Vart.append(Vart[-1]), gammat_1.append(gammat_1[-1]), gammat_2.append(gammat_2[-1])
rt.append(rt[-1])
MCRt.append(MCRt[-1]), SCRt.append(SCRt[-1])
MCRt_CF2.append(MCRt_CF2[-1]), SCRt_CF2.append(SCRt_CF2[-1])
MCRt_CF3.append(MCRt_CF3[-1]), SCRt_CF3.append(SCRt_CF3[-1])


(None, None)

2. Policyholder underwrite contract #1 (1, 60, "MARSEILLE-MARIGNANE", 5, 10)

In [39]:
S, T, station, Q_bar, l = 1, 60, "MARSEILLE-MARIGNANE", 5, 10
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

10 10 0.8069266858032 2.7236255582900277 100.88761935438352


3. Policyholder underwrite contract #2 (2, 60, "STRASBOURG-ENTZHEIM", 5, 10)

In [40]:
S, T, station, Q_bar, l = 2, 60, "STRASBOURG-ENTZHEIM", 5, 10
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

20 20 1.71787008052176 3.9620389019512277 101.88965708857393


4. Policyholder underwrite contract #3 (3, 45, "MARSEILLE-MARIGNANE", 5, 20)

In [41]:
S, T, station, Q_bar, l = 3, 45, "MARSEILLE-MARIGNANE", 5, 20
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

40 40 3.398488634852535 6.818038179860063 103.73833749833778


5. Policyholder underwrite contract #4 (4, 80 , "STRASBOURG-ENTZHEIM", 5, 5)

In [42]:
S, T, station, Q_bar, l = 4, 80, "STRASBOURG-ENTZHEIM", 5, 5
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

45 45 3.9190026921898147 6.986936383339282 104.31090296140879


6. Policyholder underwrite contract #5 (5, 102, "STRASBOURG-ENTZHEIM", 5, 15)

In [43]:
S, T, station, Q_bar, l = 5, 102, "STRASBOURG-ENTZHEIM", 5, 15
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

33.29350863099696 8.405551243812331 5.703081148761343 8.508555749021129 106.27338926363747


7. Policyholder underwrite contract #6 (6, 206, "STRASBOURG-ENTZHEIM", 5, 5)

In [44]:
S, T, station, Q_bar, l = 6, 206, "STRASBOURG-ENTZHEIM", 5, 10
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

34.18525141500301 8.983211783260714 7.179258364948983 9.218361780841832 107.89718420144388


8. Policyholder underwrites contract #7 (7, 206, "STRASBOURG-ENTZHEIM", 5, 10)

In [45]:
S, T, station, Q_bar, l = 7, 206, "STRASBOURG-ENTZHEIM", 5, 5
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

36.43702270202523 9.758983121548946 7.917346973042803 10.035264578367704 108.70908167034708


9. Policyholder underwrites contract #8 (8, 300, "MARSEILLE-MARIGNANE", 5, 15)

In [46]:
S, T, station, Q_bar, l = 8, 300, "MARSEILLE-MARIGNANE", 5, 15
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

38.95392952359113 10.70696259827682 9.641844393542804 11.11755841504916 110.60602883289708


10. Policyholder underwrites contract #9 (9, 282, "STRASBOURG-ENTZHEIM", 5, 5)

In [47]:
S, T, station, Q_bar, l = 9, 282, "STRASBOURG-ENTZHEIM", 5, 5
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

39.033515304455385 10.763900590869971 10.196578261559992 11.227913643584502 111.21623608771598


11. Policyholder underwrites contract #10 (10, 180, "MARSEILLE-MARIGNANE", 5, 20)

In [48]:
S, T, station, Q_bar, l = 10, 180, "MARSEILLE-MARIGNANE", 5, 20
add_contract(S, T, station, Q_bar, l)
print(SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

41.65250424464886 11.477102707115813 11.133819558158391 11.997185064493653 112.24720151397422


12. Settle contract #3 with $Q_{obs} = 0$

In [49]:
remove_contract(3, 0)
print(Xt[-1], SCRt_CF3[-1], MCRt_CF3[-1],Mut[-1], np.sqrt(Vart[-1]), Bt[-1])

101.84868040976386 37.292833621102744 10.221881637178864 9.453201003827616 10.636943034079618 112.24720151397422


13. Investor burns 35 tokens

In [50]:
y = 35
x = y / rt[-1] 
Yt.append(Yt[-1] - y), Xt.append(Xt[-1] - x), Bt.append(Bt[-1] - x), At.append(At[-1]), TotalSupply.append(TotalSupply[-1] - y)
Nt.append(Nt[-1]), Mt.append(Mt[-1])
Mut.append(Mut[-1]), Vart.append(Vart[-1]), gammat_1.append(gammat_1[-1]), gammat_2.append(gammat_2[-1])
rt.append(rt[-1])
MCRt.append(MCRt[-1]), SCRt.append(SCRt[-1])
MCRt_CF2.append(MCRt_CF2[-1]), SCRt_CF2.append(SCRt_CF2[-1])
MCRt_CF3.append(MCRt_CF3[-1]), SCRt_CF3.append(SCRt_CF3[-1])
print(Xt[-1], SCRt_CF3[-1], MCRt_CF3[-1], rt[-1])

67.48397400494265 37.292833621102744 10.221881637178864 1.0184868040976385


14. Settle contract # 1, 2, 4, 5, 10

In [34]:
remove_contract(1, 6)
print(Xt[-1], SCRt_CF3[-1], MCRt_CF3[-1], rt[-1])
remove_contract(2, 6)
print(Xt[-1], SCRt_CF3[-1], MCRt_CF3[-1], rt[-1])
remove_contract(4, 6)
print(Xt[-1], SCRt_CF3[-1], MCRt_CF3[-1], rt[-1])
remove_contract(5, 0)
print(Xt[-1], SCRt_CF3[-1], MCRt_CF3[-1], rt[-1])
portfolio



58.37159335932617 36.66803718345647 9.937784755422646 0.8980245132204026
49.373631093516586 35.98697725754467 9.606857332878624 0.7595943245156398
44.946196556587594 35.919658875611205 9.538223674262346 0.6914799470244245
46.90868285881628 55 55 0.7216720439817889


,contract_ID,S,T,station,Q_bar,l,theta
0,6,6,206,STRASBOURG-ENTZHEIM,5,10,0.147618
1,7,7,206,STRASBOURG-ENTZHEIM,5,5,0.147618
2,8,8,300,MARSEILLE-MARIGNANE,5,15,0.114966
3,9,9,282,STRASBOURG-ENTZHEIM,5,5,0.110947
4,10,10,180,MARSEILLE-MARIGNANE,5,20,0.046862
